In [1]:
import csv
import pandas as pd
import numpy as np
import os
import codecs
from lxml import etree

### Parsing the XML data (not useful in Hadoop env)

In [33]:
def processXML(xml):
    data=[]
    areas = xml.xpath('//histform/tellingskrets/bosted_land')
    for place in areas:
        placeName = place.findall("bostedets_navn")[0].text
        for person in place:    
            properties = [c.text for c in person if (c.tag == "fornavn" or c.tag == "patronymikon" or c.tag == "fodselsaar")]
            if len(properties) > 0:
                data.extend([[placeName] + properties])
    #d = pd.read_csv(":".join(data).replace(":","\n"), names=["Place", "Name", "Birth Year"])
    #d = pd.DataFrame(data,columns=['Place', 'Name', 'Birthyear'])
    #print(d)

def processFiles():
    for (dirpath, dirnames, filenames) in os.walk('./data/'):
        for file in filenames:
            processXML(etree.parse("{}{}".format(dirpath,file)))
        break

In [34]:
#processFiles()

### Simulate Hadoop Stream

In [2]:
stream = None
with codecs.open('./data/f8010111.xml', encoding='utf-8') as f:
    stream = f.readlines()

In [19]:
person, bosted, fodested, fodeaar, kjonn, temp = False, '?', '?', '?', '?', []

for line in stream:
    sted_start, sted_end = line.find("<bostedets_navn>"), line.find("</bostedets_navn>")
    if sted_start >= 0:
        bosted = line[(sted_start+16):sted_end]
    
    if not person:
        if line.find("<person") >= 0:
            person = True
            fodested = fodeaar = kjonn = '?'
    else:
        # We reached the end for this person, reset.
        if line.find("</person>") >= 0:
            person = False
            temp.append([bosted,fodeaar,kjonn,fodested])
            continue
            
        k = line.find("<kjonn>")
        if k >= 0:
            kjonn = line[(k+7):(k+8)]
        
        aar_start, aar_end = line.find(">"), line.find("</fodselsaar>")
        if aar_end >= 0:
            fodeaar = line[(aar_start+1):aar_end][:4]
            
        sted_start, sted_end = line.find("<fodested>"), line.find("</fodested>")
        if sted_start >= 0:
            fodested = line[(sted_start+10):sted_end]
            
            

In [20]:
temp # Use this to generate CSV (DISC: fodested is not present in pre 1910 census..)

[['Rove Præstegaard', '1764', 'm', '?'],
 ['Rove Præstegaard', '1769', 'k', '?'],
 ['Rove Præstegaard', '1795', 'k', '?'],
 ['Rove Præstegaard', '1767', 'k', '?'],
 ['Rove Præstegaard', '1775', 'k', '?'],
 ['Rove Præstegaard', '1781', 'm', '?'],
 ['Rove Præstegaard', '1739', 'm', '?'],
 ['Rove Præstegaard', '1755', 'k', '?'],
 ['Rove Præstegaard', '1789', 'm', '?'],
 ['Rove Præstegaard', '1793', 'm', '?'],
 ['Rove Præstegaard', '1784', 'k', '?'],
 ['Rove Præstegaard', '1797', 'k', '?'],
 ['Rove Præstegaard', '1740', 'm', '?'],
 ['Rove Præstegaard', '1757', 'k', '?'],
 ['Rove Præstegaard', '1776', 'm', '?'],
 ['Rove Præstegaard', '1792', 'm', '?'],
 ['Rove Præstegaard', '1798', 'm', '?'],
 ['Rove Præstegaard', '1771', 'm', '?'],
 ['Rove Præstegaard', '1778', 'k', '?'],
 ['Rove Præstegaard', '1800', 'k', '?'],
 ['Rove Præstegaard', '1790', 'k', '?'],
 ['Rove Præstegaard', '1741', 'm', '?'],
 ['Rove Præstegaard', '1745', 'k', '?'],
 ['Rove Præstegaard', '1747', 'm', '?'],
 ['Rove Præstega